# Simulation API

A merchant wants to offer products and maximize profits. Just like on a real online marketplace, he can look at all existing offers, add some own, restock or reprice. She has to buy (or produce) products first, which comes with costs. This is partially done by the producer. The entities are all implemented as services and their interfaces (REST) are described in detail [here](https://hpi-epic.github.io/masterproject-pricewars/).

This notebook will present how to use the Merchant SDK to do all these tasks easily. From registration to buying products, offering them and repricing them. Everything else is left to the merchant logic.

*Note*: The SDK is type-hinted, so using an IDE (e.g. PyCharm/IntelliJ or an IPython/Jupyter notebook) provides you with auto-completion (that is also the reason why Python >= 3.5 is required).

## Install requirements

Get the `merchant_sdk` folder, python >= 3.5 and install the dependencies using pip:

```
cd merchant_sdk
pip3 install -r requirements.txt
```

## Import SDK

In [3]:
import sys
sys.path.append('../../')

import merchant_sdk

## Init API

make sure to have all services running by either deploying them individually or using the docker and docker-compose files.

In [7]:
from merchant_sdk.api import MarketplaceApi, ProducerApi, PricewarsRequester

In [5]:
marketplace_endpoint = 'http://vm-mpws2016hp1-04.eaalab.hpi.uni-potsdam.de:8080/marketplace'
marketplace_api = MarketplaceApi(host=marketplace_endpoint)

producer_endpoint = 'http://vm-mpws2016hp1-03.eaalab.hpi.uni-potsdam.de'
producer_api = ProducerApi(host=producer_endpoint)

## Register as merchant

In order to act on the marketplace, we need to be a registered merchant. Usually you use the Management UI to register a merchant and remember/keep the merchant_token. However, you can also call the API with the SDK.

You will also have to provie an API endpoint for your merchant, which will be called upon sales of products. We will simply use an invalid one, since this is only an example.

In [6]:
registration = marketplace_api.register_merchant(
    api_endpoint_url='http://nobody:55000/',
    merchant_name='sample_merchant',
    algorithm_name='human')

registration

{"api_endpoint_url": "http://nobody:55000/", "merchant_name": "sample_merchant", "algorithm_name": "human", "merchant_id": "6ma2rJrhyXtGQZYvSECgCQLs2ctmNFKDwNrJd8gmMs0=", "merchant_token": "Y0kJF3j7sinCJS9VgFLwCC1sHnQd9Y3hYn1nqkujSivlB6HqLcnXopQ6crjlQmTy"}

## Setup Authentication

Most of the APIs and actions a merchant can do are secured by the merchants token. Imagine in a competitive simulation, other teams/merchant should not be able to act on your behalf.

In [8]:
PricewarsRequester.add_api_token(registration.merchant_token)

## Check offers on the market

If you want to program your merchant state-less, you can set `include_empty_offers` to True. This will add your own, but out-of-stock offers to be added to the response.

A sucessful response will be casted to the according model.

In [10]:
offers = marketplace_api.get_offers(include_empty_offers=False)
offers[:2]

[{"amount": 4, "merchant_id": "XUxM+XmfkUI2ZeZ2Mphwf066k46Y17X5QaiqDioocZo=", "offer_id": 7510, "price": 16.5, "prime": true, "product_id": 4, "quality": 1, "shipping_time": {"standard": 5, "prime": 1}, "signature": "", "uid": 41},
 {"amount": 1, "merchant_id": "XUxM+XmfkUI2ZeZ2Mphwf066k46Y17X5QaiqDioocZo=", "offer_id": 7528, "price": 13.2, "prime": true, "product_id": 4, "quality": 2, "shipping_time": {"standard": 5, "prime": 1}, "signature": "", "uid": 42}]

In [11]:
type(offers[0])

merchant_sdk.models.Offer.Offer

## Buy products

At the time, writing this sample/notebook, the producer did not support the Authorization Token in the HTTP header, so I will have to explicitly add it as a parameter.

In [13]:
product = producer_api.buy_product(merchant_token=registration.merchant_token)
product

{"amount": 1, "name": "CD_4", "price": 12, "product_id": 4, "quality": 2, "signature": "MTjIT366xPxgHMuCIiwwBVv84OwFDT2i1w4+Tl4L7IxrBBLd1xC0pei6lqNVKW3MUkXfyev2xg9MLGrqTvw7jiDAkX3lac58YJub1bUlmQK1sVVPMiQXUUFPIkzpT2YhtbFVTzIkF1FBTyJM6U9mIbWxVU8yJBdRQU8iTOlPZiE=", "uid": 42, "stock": -1, "time_to_live": -1, "start_of_lifetime": -1}

In [14]:
type(product)

merchant_sdk.models.Product.Product

## Add product to marketplace

Products need to be wrapped in an offer in order to put them on the market.

You need to provide the latest product signature to the offer, so that the marketplace can confirm the validity of the offer and its product.

You can then mutate/adapt the offer and submit it to the market.

In [15]:
new_offer = merchant_sdk.models.Offer.from_product(product)
new_offer

{"amount": 1, "merchant_id": "", "offer_id": -1, "price": 12, "prime": false, "product_id": 4, "quality": 2, "shipping_time": {"standard": 3}, "signature": "MTjIT366xPxgHMuCIiwwBVv84OwFDT2i1w4+Tl4L7IxrBBLd1xC0pei6lqNVKW3MUkXfyev2xg9MLGrqTvw7jiDAkX3lac58YJub1bUlmQK1sVVPMiQXUUFPIkzpT2YhtbFVTzIkF1FBTyJM6U9mIbWxVU8yJBdRQU8iTOlPZiE=", "uid": 42}

In [16]:
type(new_offer)

merchant_sdk.models.Offer.Offer

In [17]:
margin = 0.50
new_offer.price *= 1.0 + margin
new_offer

{"amount": 1, "merchant_id": "", "offer_id": -1, "price": 18.0, "prime": false, "product_id": 4, "quality": 2, "shipping_time": {"standard": 3}, "signature": "MTjIT366xPxgHMuCIiwwBVv84OwFDT2i1w4+Tl4L7IxrBBLd1xC0pei6lqNVKW3MUkXfyev2xg9MLGrqTvw7jiDAkX3lac58YJub1bUlmQK1sVVPMiQXUUFPIkzpT2YhtbFVTzIkF1FBTyJM6U9mIbWxVU8yJBdRQU8iTOlPZiE=", "uid": 42}

In [18]:
new_offer = marketplace_api.add_offer(new_offer)
new_offer

{"amount": 1, "merchant_id": "6ma2rJrhyXtGQZYvSECgCQLs2ctmNFKDwNrJd8gmMs0=", "offer_id": 7769, "price": 18.0, "prime": false, "product_id": 4, "quality": 2, "shipping_time": {"standard": 3}, "signature": "", "uid": 42}

In [20]:
[o for o in marketplace_api.get_offers() if o.offer_id == new_offer.offer_id][0]

{"amount": 1, "merchant_id": "6ma2rJrhyXtGQZYvSECgCQLs2ctmNFKDwNrJd8gmMs0=", "offer_id": 7769, "price": 18.0, "prime": false, "product_id": 4, "quality": 2, "shipping_time": {"standard": 3}, "signature": "", "uid": 42}

## Update product on marketplace

Updating a product, e.g. changing its price, is a limited API request. According to your simulation/marketplace settings, we can only call it N times per second.

In [23]:
new_offer.price += 10
marketplace_api.update_offer(new_offer)

In [24]:
[o for o in marketplace_api.get_offers() if o.offer_id == new_offer.offer_id][0]

{"amount": 1, "merchant_id": "6ma2rJrhyXtGQZYvSECgCQLs2ctmNFKDwNrJd8gmMs0=", "offer_id": 7769, "price": 28.0, "prime": false, "product_id": 4, "quality": 2, "shipping_time": {"standard": 3}, "signature": "", "uid": 42}

## Unregister the merchant

You should keep your merchant and the token as long as possible, because it is the reference to all market data (sales, profit, marketshare), offers and products.

However, if you just try things out, like in this sample and don't want to pollute the database with lots of merchants, unregister it. This also removes all offers and products.

In [25]:
marketplace_api.unregister_merchant(merchant_token=registration.merchant_token)